In [1]:
#uruchomienie sesji SPARKA
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import functions
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1674897189649_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
Liczba postów na przestrzeni czasu
dfPosts = spark.read.parquet('s3://stackexchangebeer1/parquet/posts.parquet')
df = dfPosts.groupBy('CreationDate').agg(f.count("CreationDate").alias("NumberOfPosts"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%%display
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [11]:
#Porównanie najwyżej i najniżej ocenianych pytań
dfPost1 = dfPosts.select("AnswerCount","Tags","Title","Score")
dfPostNoNull = poster.dropna("any")
df = dfPostNoNull.select("Title","Tags","AnswerCount","Score").sort(f.col("Score").desc()).show(5,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------+----------------------------------------------+-----------+-----+
|Title                                                            |Tags                                          |AnswerCount|Score|
+-----------------------------------------------------------------+----------------------------------------------+-----------+-----+
|Do different beer glass shapes really make a difference in taste?|glassware taste aroma                         |5          |68   |
|Do cans change the taste of beer?                                |storage cans                                  |9          |46   |
|Where can I find open APIs about beer?                           |breweries                                     |3          |45   |
|How can I tell whether a beer will improve with age?             |aging bottle-conditioning cellaring freshness |8          |43   |
|What is the difference between an ale and a lager?               |br

In [12]:
#Porównanie najwyżej i najniżej ocenianych pytań
dfPostNoNull.createOrReplaceTempView("DfPosts")
spark.sql("select Title,Tags,AnswerCount,Score from DfPosts ORDER BY Score asc").show(5,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------+---------------------------------------------------+-----------+-----+
|Title                                           |Tags                                               |AnswerCount|Score|
+------------------------------------------------+---------------------------------------------------+-----------+-----+
|Why are breweries against freezing pint glasses?|temperature glassware                              |4          |-7   |
|What is your favorite beer & food pairing?      |pairing                                            |3          |-3   |
|Beer recommendations for a noob                 |taste recommendations breweries flavor craft-beers |3          |-3   |
|Does a Sober Scottish man really exists?        |drinking craft-beers scotch                        |1          |-3   |
|Cheapest Gin in the UK?                         |spirits gin online                                 |4          |-2   |
+-------------------------------

In [9]:
#Top N tagów które wygenerowały najwięcej wyświetleń
dfTags = spark.read.parquet('s3://stackexchangebeer1/parquet/tags.parquet')
dfTags1 = dfTags.select("TagName","Count")
dfTags1.createOrReplaceTempView("DfTags")
df = spark.sql("select TagName, Count as Views from DfTags ORDER BY Count desc limit 5")
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+
|        TagName|Views|
+---------------+-----+
|           wine|  137|
|          taste|  120|
|        history|   85|
|recommendations|   78|
|        brewing|   69|
+---------------+-----+

In [8]:
%%display
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [14]:
dfTags.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Count: integer (nullable = true)
 |-- PostId: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- TagName: string (nullable = true)
 |-- WikiPostId: integer (nullable = true)

In [27]:
#najczęściej występujące słowa w tytułach

#funkcja usuwająca niepotrzebne znaki oraz zmniejszająca wszystkie litery
from pyspark.sql.functions import regexp_replace, trim, col, lower

def remove_chars(column):
    return trim(lower(regexp_replace(column, '([^\s\w_]|_)+', ''))).alias('Title')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#aplikacja funkcji do DataFrame'u składjącego się z kolumny Title
df = dfPosts.select("Title").select(remove_chars(col('Title')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df1 = df.na.drop("all")

In [29]:
#podzielenie słów oraz ich podliczenie
from pyspark.sql import functions as f 

df.withColumn('word', f.explode(f.split(f.col('Title'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+
| word|count|
+-----+-----+
| beer|  372|
|  the|  330|
|    a|  309|
| what|  255|
|   in|  247|
|   to|  245|
|   is|  243|
|   of|  231|
|  how|  147|
|  are|  142|
|  and|  142|
| wine|  128|
|  for|  125|
|  why|  111|
|   do|  105|
|  can|  104|
|beers|  100|
|    i|   90|
| with|   82|
|there|   82|
+-----+-----+
only showing top 20 rows

In [9]:
stopwords = ['i','me','my','myself','we','our','ours','ourselves','you',"you're","you've","you'll","you'd",'your','yours','yourself','yourselves','he','him',\
 'his','himself','she',"she's",'her','hers','herself','it',"it's",'its','itself','they','them','their','theirs','themselves','what','which','who','whom','this','that',\
 "that'll",'these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because',\
 'as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','down','in','out','on',\
 'off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both','each','few','more','most','other','some','such','no','nor','not',\
 'only','own','same','so','than','too','very','s','t','can','will','just','don',"don't",'should',"should've",'now','d','ll','m','o','re','ve','y','ain','aren',"aren't",'couldn',\
 "couldn't", 'didn',"didn't",'doesn',"doesn't",'hadn',"hadn't",'hasn',"hasn't",'haven',"haven't",'isn',"isn't",'ma','mightn',"mightn't",'mustn',"mustn't",'needn', \
 "needn't",'shan', "shan't",'shouldn',"shouldn't",'wasn',"wasn't",'weren',"weren't",'won',"won't",'wouldn',"wouldn't", ' ']

df2.filter(~df2['word'].isin(stopwords)).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#zaimportowanie tabeli Posts oraz Users
dfPosts = spark.read.parquet('s3://stackexchangebeer1/parquet/posts.parquet')
dfUsers = spark.read.parquet('s3://stackexchangebeer1/parquet/users.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
#procent użytkowników którzy nigdy nic nie zapostowali

#liczba uczestników która stworzyła kiedykolwiek post
dfUsers1 = dfUsers.select("Id")
dfPosts1 = dfPosts.select("OwnerUserId")
num_of_user_create_post = dfUsers1.join(dfPosts1, dfUsers1.Id == dfPosts1.OwnerUserId, "inner").distinct().count()

#całkowita liczby użytkowników
num_of_users = dfUsers.count()

#% użytkowników bez stworzenia posta
percent_of_users_without_post = 100 - round((num_of_user_create_post / num_of_users),4)*100
print(f'Udzial uzytkownikow ktorzy nigdy nic nie zapostowali wynosi {percent_of_users_without_post}%')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Udzial uzytkownikow ktorzy nigdy nic nie zapostowali wynosi 86.35%

In [4]:
#porównanie punktacji odpowiedzi zaakceptowanych vs pozostał

dfComments = spark.read.parquet('s3://stackexchangebeer1/parquet/comments.parquet')
dfVotes = spark.read.parquet('s3://stackexchangebeer1/parquet/votes.parquet')

dfVotes1 = dfVotes.select("PostId", "VoteTypeId")
dfComments1 = dfComments.select("PostId", "Score")

df = dfVotes1.join(dfComments1, dfVotes1.PostId == dfComments1.PostId, "inner")

df.createOrReplaceTempView("dfPostsComments")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
spark.sql("select max(Score) as MaxScore , min(Score) as MinScore , round(avg(Score),2) as AvgScore, \
          round(stddev(Score),2) as StdScore from dfPostsComments where VoteTypeId = 1").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+--------+--------+
|MaxScore|MinScore|AvgScore|StdScore|
+--------+--------+--------+--------+
|      12|       0|    0.45|    1.08|
+--------+--------+--------+--------+

In [6]:
spark.sql("select max(Score) as MaxScore , min(Score) as MinScore , round(avg(Score),2) as AvgScore, \
          round(stddev(Score),2) as StdScore from dfPostsComments where VoteTypeId != 1").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+--------+--------+
|MaxScore|MinScore|AvgScore|StdScore|
+--------+--------+--------+--------+
|      19|       0|    0.65|    1.41|
+--------+--------+--------+--------+

In [ ]:
#wykresy dla top 5 tagów
sc.install_pypi_package("matplotlib==3.1.1", "https://pypi.org/simple") 
sc.install_pypi_package("pandas==0.25.1")

In [ ]:
#zaimportowanie tabeli Posts oraz jej wyczyszczenie i uporządkowanie
from pyspark.sql import functions as f
dfPosts = spark.read.parquet('s3://stackexchangebeer1/parquet/posts1.parquet')
dfPosts1 = dfPosts.select("CreationDate", "Tags")
df2 = dfPosts1.withColumn('Tags',f.regexp_replace(f.regexp_replace('Tags', '<', ''),'>', ' '))
df3 = df2.withColumn("Tags",f.expr("substring(Tags, 1, length(Tags)-1)"))
df4 = df3.withColumn('Tags', f.explode(f.split(f.col('Tags'), ' ')))
df4.show(10)

In [ ]:
df_brewing = df4.filter(df4.Tags == 'brewing')
tag_brewing = df_brewing.groupBy('CreationDate').agg(f.count('CreationDate').alias('brewing'))

df_history = df4.filter(df4.Tags == 'history')
tag_history = df_history.groupBy('CreationDate').agg(f.count('CreationDate').alias('history'))

df_reccomendations = df4.filter(df4.Tags == 'reccomendations')
reccomendations_wine = df_reccomendations.groupBy('CreationDate').agg(f.count('CreationDate').alias('reccomendations'))

df_taste = df4.filter(df4.Tags == 'taste')
tag_taste = df_taste.groupBy('CreationDate').agg(f.count('CreationDate').alias('taste'))

df_wine = df4.filter(df4.Tags == 'wine')
tag_wine = df_wine.groupBy('CreationDate').agg(f.count('CreationDate').alias('wine'))



In [ ]:
%%display
tag_twine

In [ ]:
import matplotlib.pyplot as plt
import pandas
tag_wine_pand = tag_wine.withColumn("wine", tag_wine["wine"].cast('int'))
df = tag_wine_pand.toPandas()
plt.clf()
plt.bar(df.CreationDate,df.wine)
plt.xlabel('CreationDate')
plt.ylabel('wine')
plt.show()
%matplot plt

In [ ]:
import matplotlib.pyplot as plt
import pandas
tag_taste_pand = tag_taste.withColumn("taste", tag_taste["taste"].cast('int'))
df = tag_taste_pand.toPandas()
plt.clf()
plt.bar(df.CreationDate,df.taste)
plt.xlabel('CreationDate')
plt.ylabel('taste')
plt.show()
%matplot plt

In [ ]:
import matplotlib.pyplot as plt
import pandas
tag_recommendations_pand = tag_recommendations.withColumn("recommendations", tag_recommendations["recommendations"].cast('int'))
df = tag_recommendations_pand.toPandas()
plt.clf()
plt.bar(df.CreationDate,df.recommendations)
plt.xlabel('CreationDate')
plt.ylabel('reccomendations')
plt.show()
%matplot plt

In [ ]:
import matplotlib.pyplot as plt
import pandas
tag_history_pand = tag_history.withColumn("history", tag_history["history"].cast('int'))
df = tag_history_pand.toPandas()
plt.clf()
plt.bar(df.CreationDate,df.history)
plt.xlabel('CreationDate')
plt.ylabel('history')
plt.show()
%matplot plt

In [ ]:
import matplotlib.pyplot as plt
tag_brewing_pand = tag_brewing.withColumn("brewing", tag_brewing["brewing"].cast('int'))
plt.clf()
df = tag_brewing_pand.toPandas()
plt.bar(df.CreationDate,df.brewing)
plt.xlabel('CreationDate')
plt.ylabel('brewing')
plt.figure(figsize=(10,6))
plt.show()
%matplot plt

In [ ]:
#średni czas pojawienia się zaakceptowanej odpowiedzi
dfPosts = spark.read.parquet('s3://stackexchangebeer1/parquet/posts.parquet')
dfVotes = spark.read.parquet('s3://stackexchangebeer1/parquet/votes.parquet')
dfComments = spark.read.parquet('s3://stackexchangebeer1/parquet/comments.parquet')

#wybranie odpowiednich kolumn z tabel i zmiana nazwy kolumn
dfComments1 = dfComments.select("PostId", "Date")
dfComments2 = dfComments1.withColumnRenamed("Date","CommentDate")
dfPosts1 = dfPosts.select('Id', 'Date')
dfVotes1 = dfVotes.select('PostId', 'VoteTypeId')

#połączenie funkcją join tabel Posts oraz Votes
dfmerge = dfPosts1.join(dfVotes2, dfPosts1.Id == dfVotes2.PostId, 'inner')

#połączenie funkcją join tabli Comments z wcześniej połączoną tabelą Posts oraz Votes
df = dfmerge.join(dfComments2, dfmerge.PostId == dfComments2.PostId, 'inner')

In [ ]:
from pyspark.sql import functions as f

#przefiltrowanie zaakceptowanych odpowiedzi
df_answer_time = df.filter(df.VoteTypeId == 1)

#obliczenie czasu pojawienia się zaakceptowanej odpowiedzi w sekundach i dniach
df_time = df_answer_time.withColumn("diffinsec", f.col("CommentDate").cast("long") - f.col("Date").cast("long"))
df_time = df_time.withColumn("diffindays", f.col("diffinsec")/(3600*24))

df_time.createOrReplaceTempView("DF")
df = spark.sql("select round(avg(diffindays),2) as avg_time_days from DF")
df.show()

In [ ]:
#najdłużej istniejący użytkownicy na forum StackExchange
dfComments = spark.read.parquet('s3://stackexchangebeer1/parquet/comments.parquet')
dfUsers = spark.read.parquet('s3://stackexchangebeer1/parquet/users.parquet')

#wybranie odpowiednich kolumn do analizy
dfUsers1 = dfUsers.select("Id", "CreationDate","DisplayName")
dfComments1 = dfComments.select("UserId", "Date")
dfComments2 = dfComments1.withColumnRenamed("Date","CommentDate")

In [ ]:
#połączenie kolumn Comments oraz Users
df_merge = dfUsers1.join(dfComments2, dfUsers1.Id == dfComments2.UserId, 'inner')

#policzenie różnicy czasu stworzenia konta użytkownika do jego ostatniego komentarza
df_time = df_merge.withColumn("diffinsec", f.col("CommentDate").cast("long") - f.col("CreationDate").cast("long"))
df_time = df_time.withColumn("diffindays", f.col("diffinsec")/(3600*24))

In [ ]:
#usunięcie użytkownika z Id = -1 - jest to froum_bot
df_life_time = df_time.filter(df_time.Id != -1)

#stworzenie TempView i przefiltorawanie wyników po nazwach użytkowników oraz najdłuższym czasie istnienia
df_life_time.createOrReplaceTempView("time")
df_life = spark.sql("select distinct DisplayName, round(max(diffindays),2) as life_time_day from time group by DisplayName order by life_time_day desc limit 10")
df_life.show()

In [ ]:
#wykres najdłużej istniejących użytkowników
%%display 
df_life

In [ ]:
#procent zaakecptowanych odpoweiedzi oraz odpowiedzi nieezakceptowanych z najwyższą punktacją

#dobranie odpowiednich kolumn do analizy
dfComments1 = dfComments.select("PostId", "Score")
dfVotes1 = dfVotes.select("PostId", "VoteTypeId")
dfVotes2 = dfVotes1.withColumnRenamed("PostId","Id")

#połączenie kolumn
dfmerge = dfComments1.join(dfVotes2, dfComments1.PostId == dfVotes2.Id, 'inner')
#policzenie wszystkich komenatrzy
all_comments = dfmerge.count()

In [ ]:
#obliczenie ilości odpowiedzi z maksymalnym wynikiem które nie został zaakceptowane
dfmerge.createOrReplaceTempView("df")
df = spark.sql("select distinct PostId, max(score) as score from df where VoteTypeId != 1 group by PostId")
df_max_score_not_accepted = df.filter(df.score > 0)
max_score = df_max_score_not_accepted.count()

In [ ]:
#obliczenie ilości zaakceptowanych odpowiedzi
df_accepted_answer = spark.sql("select  PostId, Score as score from df where VoteTypeId = 1")
accepted = df_accepted_answer.count()

In [ ]:
x = round((766/27106 *100),2)
y =round((838/27106 *100),2)

print(f'Ilosc zaakceptowanych komentarzy wzgledem wszysktkich komenatrzy wynosi {y}%, zas komentarze nieezaakceptowane z najwyzsza punktacja stanowia {y}%')